# SQL query with Py Pandas #
This notebook demonstrates some data manipulating practice using SQL queries and as a support I will use sometimes Python Pandas functions. With two different options to manipulate the data it's become easy to compere the results.

This noutbook contains 4 main sections:
1. SELECT... WHERE...ORDER (ASC/DESC)
2. JOIN (INNER, OUTER, RIGHT, LEFT)
3. GROUP BY + Aggregation functions
4. CTE (WITH)
5. Bonus (Rank() vs Row_number())

Database with name **'mydatabase'** has been already loaded to SQL server and this data has two tables **'myrun'** and **'myrun_HR'**. It's information about my runnings for last 9 years and the table **'myrun'** has 1289 records in four columns:
- id - index
- RunDt - Date & Time of Running
- RunTt - Location
- RunDs - Running distance in km
- RunTm - Running time

table **'myrun_HR'** - it's information about heart rate during the running, we need it for JOIN practice and it has only 387 records in in 3 columns:

- RunDt_HR - Date & Time of Running (related to table **'myrun'**)
- RunDs_HR - (related to table **'myrun'**)
- Avg_HR - new data

Let's play with data...

In [1]:
import mysql.connector
import pandas as pd
mydb = mysql.connector.connect(host='localhost',
                               user='yukochu',  
                               passwd='111111',  
                               database='mydatabase')
mycrs = mydb.cursor()

## 1 . SELECT ... WHERE... ORDER
1.1. For the beginning let's open whole table. Pandas DataFrame **'df'** going to be permanent data frame for compering results. Lat's take a look to the data itself:

In [2]:
que = 'SELECT * FROM myrun'
mycrs.execute(que)
datarequest = mycrs.fetchall()
df = pd.DataFrame(datarequest, columns=mycrs.column_names) 
df['RunDs'] = df['RunDs'].astype('float64')
df #------------------------- Pandas

,id,RunDt,RunTt,RunDs,RunTm
0,1,1/5/2011 0:00,Odesa,6.00,0:35:00
1,2,1/6/2011 0:00,Odesa,6.00,0:32:00
2,3,1/7/2011 0:00,Odesa,6.00,0:32:00
3,4,1/10/2011 0:00,Odesa,5.22,0:26:00
4,5,1/11/2011 0:00,Odesa,5.22,0:26:00
...,...,...,...,...,...
1184,1185,12/6/2019 10:00,Run,5.00,0:27:40
1185,1186,12/6/2019 13:34,Odesa Running,8.20,0:49:58
1186,1187,12/10/2019 9:26,Odesa Running,6.05,0:35:02
1187,1188,12/11/2019 13:34,Odesa Running,6.18,0:35:03


1.2. Select only a couple columns considering only first 10 rows


In [3]:
que = 'SELECT RunDt, RunDs FROM myrun'
mycrs.execute(que)
datarequest = mycrs.fetchall()
print(datarequest[:10])
df[['RunDt', 'RunDs']].head(10) #------------------------- Pandas

[('1/5/2011 0:00', '6'), ('1/6/2011 0:00', '6'), ('1/7/2011 0:00', '6'), ('1/10/2011 0:00', '5.22'), ('1/11/2011 0:00', '5.22'), ('1/13/2011 0:00', '5.22'), ('1/14/2011 0:00', '5.22'), ('1/15/2011 0:00', '5.22'), ('1/16/2011 0:00', '10.7'), ('1/18/2011 0:00', '7.32')]


,RunDt,RunDs
0,1/5/2011 0:00,6.00
1,1/6/2011 0:00,6.00
2,1/7/2011 0:00,6.00
3,1/10/2011 0:00,5.22
4,1/11/2011 0:00,5.22
5,1/13/2011 0:00,5.22
6,1/14/2011 0:00,5.22
7,1/15/2011 0:00,5.22
8,1/16/2011 0:00,10.70
9,1/18/2011 0:00,7.32


1.3. Select Runnings longer than 35km

In [4]:
que = 'SELECT RunTt, RunDs FROM myrun WHERE RunDs>35'
mycrs.execute(que)
datarequest = mycrs.fetchall()
print(datarequest[:10])
tempDF = df[['RunTt', 'RunDs']]#------------------------- Pandas
tempDF[tempDF.RunDs>35].head(10)#------------------------- Pandas

[('Odesa Ultra Marathon', '100'), ('N.Novgorod Marathon', '42.2'), ('Togliatti Marathon', '42.2'), ('Brasov Marathon', '42.2')]


,RunTt,RunDs
50,Odesa Ultra Marathon,100.0
389,N.Novgorod Marathon,42.2
638,Togliatti Marathon,42.2
1092,Brasov Marathon,42.2


In [5]:
df.columns

Index(['id', 'RunDt', 'RunTt', 'RunDs', 'RunTm'], dtype='object')

1.4. Select NOT Odessa (Title starts with the word 'Odessa') Runnings where distance is between 5km and 10km

In [6]:
que = 'SELECT * FROM myrun WHERE NOT RunTt="Odesa" \
                                 AND RunDs>5 \
                                 AND RunDs<10'
mycrs.execute(que)
datarequest = mycrs.fetchall()
print(datarequest[:10])
df[(df.RunTt!='Odesa') & (df.RunDs>5)& (df.RunDs<10)].head(10)#------------------------- Pandas

[(195, '1/13/2013 0:00', 'Turkmenistan', '6.5', '0:38:00'), (202, '2/26/2013 0:00', 'Turkmenistan', '5.2', '0:27:48'), (203, '2/27/2013 11:24', 'Turkmenistan', '7.3', '0:37:52'), (233, '5/29/2013 20:15', 'Turkmenistan', '5.26', '0:30:30'), (239, '6/16/2013 19:15', 'Turkmenistan', '8.06', '0:46:34'), (243, '6/27/2013 19:11', 'Turkmenistan', '5.05', '0:29:07'), (246, '7/4/2013 19:28', 'Turkmenistan', '5.09', '0:27:01'), (253, '7/18/2013 21:02', 'Turkmenistan', '5.1', '0:30:25'), (257, '7/25/2013 19:47', 'Turkmenistan', '5.16', '0:26:52'), (260, '7/31/2013 19:51', 'Turkmenistan', '6.18', '0:36:20')]


,id,RunDt,RunTt,RunDs,RunTm
194,195,1/13/2013 0:00,Turkmenistan,6.50,0:38:00
201,202,2/26/2013 0:00,Turkmenistan,5.20,0:27:48
202,203,2/27/2013 11:24,Turkmenistan,7.30,0:37:52
232,233,5/29/2013 20:15,Turkmenistan,5.26,0:30:30
238,239,6/16/2013 19:15,Turkmenistan,8.06,0:46:34
242,243,6/27/2013 19:11,Turkmenistan,5.05,0:29:07
245,246,7/4/2013 19:28,Turkmenistan,5.09,0:27:01
252,253,7/18/2013 21:02,Turkmenistan,5.10,0:30:25
256,257,7/25/2013 19:47,Turkmenistan,5.16,0:26:52
259,260,7/31/2013 19:51,Turkmenistan,6.18,0:36:20


1.5. Select Odessa Runnings (contain 'Odesa' word) where distance was more than 40km. Lets sort it by distance with descending order

In [7]:
que = 'SELECT * FROM myrun WHERE RunTt LIKE "%Odesa%" \
                                 AND RunDs>30 \
                                 ORDER BY RunDs DESC'
mycrs.execute(que)
datarequest = mycrs.fetchall()
print(datarequest)
df[(df.RunTt.str.contains('Odesa')) & (df.RunDs>30)].sort_values('RunDs', ascending=True)#----- Pandas

[(1084, '5/12/2019 12:01', 'Odesa Running', '32.12', '3:01:45'), (1079, '5/5/2019 14:18', 'Odesa Running', '30.37', '2:51:21'), (43, '3/27/2011 0:00', 'Odesa', '30.3', '2:44:00'), (1075, '5/1/2019 11:19', 'Odesa Running', '30.11', '3:09:42'), (51, '4/9/2011 0:00', 'Odesa Ultra Marathon', '100', '10:59:00')]


,id,RunDt,RunTt,RunDs,RunTm
1074,1075,5/1/2019 11:19,Odesa Running,30.11,3:09:42
42,43,3/27/2011 0:00,Odesa,30.30,2:44:00
1078,1079,5/5/2019 14:18,Odesa Running,30.37,2:51:21
1083,1084,5/12/2019 12:01,Odesa Running,32.12,3:01:45
50,51,4/9/2011 0:00,Odesa Ultra Marathon,100.00,10:59:00


## 2 . JOIN (inner, outer, left, right) ## 

Our database has another table 'myrun_HR' with some data with Heart Rate but not for all runnings, only 387. 

- RunDt_HR - Date & Time of Running (related to table 'myrun')
- RunDs_HR - (related to table 'myrun')
- Avg_HR - new data

For both tables the columns **myrun.RunDt** and **myrun_hr.RunDt_HR** are the same (related) and the dates are unique.

In [8]:
que = 'SELECT * FROM myrun_hr'
mycrs.execute(que)
datarequest = mycrs.fetchall()
df = pd.DataFrame(datarequest, columns=mycrs.column_names) 
df

,RunDt_HR,RunDs_HR,Avg_HR
0,12/12/2019 15:11,6.07,154
1,12/11/2019 13:34,6.18,141
2,12/10/2019 9:26,6.05,148
3,12/6/2019 13:34,8.2,148
4,12/2/2019 13:35,5.91,153
...,...,...,...
382,3/22/2018 12:02,11.49,156
383,3/21/2018 14:03,11.46,160
384,3/19/2018 11:37,11.46,159
385,3/17/2018 12:23,11.4,159


In [9]:
que = 'SELECT myrun.RunDt AS dt, myrun.RunTt, myrun.RunDs, myrun_hr.Avg_HR as hr \
        FROM myrun INNER JOIN myrun_hr \
        ON myrun.RunDt=myrun_hr.RunDt_HR'
mycrs.execute(que)
datarequest = mycrs.fetchall()
df = pd.DataFrame(datarequest, columns=mycrs.column_names)
print(datarequest[:10])
print('------------')
print('Length of the new table:', len(datarequest))
df

[('3/16/2018 20:48', 'Odesa', '8.13', '156'), ('3/17/2018 12:23', 'Odesa', '11.4', '159'), ('3/19/2018 11:37', 'Odesa', '11.46', '159'), ('3/21/2018 14:03', 'Odesa', '11.46', '160'), ('3/22/2018 12:02', 'Odesa', '11.49', '156'), ('3/24/2018 9:37', 'Odesa', '11.5', '157'), ('3/29/2018 11:48', 'Yamal', '8.29', '170'), ('3/31/2018 15:45', 'Yamal', '10.34', '164'), ('4/2/2018 15:52', 'Yamal', '10.94', '165'), ('4/3/2018 16:13', 'Yamal', '5.74', '147')]
------------
Length of the new table: 308


,dt,RunTt,RunDs,hr
0,3/16/2018 20:48,Odesa,8.13,156
1,3/17/2018 12:23,Odesa,11.4,159
2,3/19/2018 11:37,Odesa,11.46,159
3,3/21/2018 14:03,Odesa,11.46,160
4,3/22/2018 12:02,Odesa,11.49,156
...,...,...,...,...
303,12/2/2019 13:35,Odesa Running,5.91,153
304,12/6/2019 13:34,Odesa Running,8.2,148
305,12/10/2019 9:26,Odesa Running,6.05,148
306,12/11/2019 13:34,Odesa Running,6.18,141


In [10]:
que = 'SELECT myrun.RunDt, myrun.RunTt, myrun.RunDs, myrun_hr.Avg_HR \
        FROM myrun LEFT JOIN myrun_hr \
        ON myrun.RunDt=myrun_hr.RunDt_HR'
mycrs.execute(que)
datarequest = mycrs.fetchall()
df = pd.DataFrame(datarequest, columns=mycrs.column_names)
print(datarequest[:10])
print('------------')
print('Length of the new table:', len(datarequest))
df

[('12/12/2019 15:11', 'Odesa Running', '6.07', '154'), ('12/11/2019 13:34', 'Odesa Running', '6.18', '141'), ('12/10/2019 9:26', 'Odesa Running', '6.05', '148'), ('12/6/2019 13:34', 'Odesa Running', '8.2', '148'), ('12/2/2019 13:35', 'Odesa Running', '5.91', '153'), ('11/30/2019 10:01', 'Odesa Running', '5.03', '160'), ('11/29/2019 13:33', 'Odesa Running', '6.07', '144'), ('11/28/2019 11:18', 'Odesa Running', '6.33', '157'), ('11/27/2019 13:28', 'Odesa Running', '6.03', '153'), ('11/26/2019 13:30', 'Odesa Running', '6.05', '154')]
------------
Length of the new table: 1189


,RunDt,RunTt,RunDs,Avg_HR
0,12/12/2019 15:11,Odesa Running,6.07,154
1,12/11/2019 13:34,Odesa Running,6.18,141
2,12/10/2019 9:26,Odesa Running,6.05,148
3,12/6/2019 13:34,Odesa Running,8.2,148
4,12/2/2019 13:35,Odesa Running,5.91,153
...,...,...,...,...
1184,3/5/2018 14:16,Poscot,11.5,None
1185,3/6/2018 15:27,poscot,11.5,None
1186,3/8/2018 19:25,poscot,11.5,None
1187,1/6/2019 9:45,Sunday,12.53,None


In [11]:
que = 'SELECT myrun.RunDt, myrun.RunTt, myrun.RunDs, myrun_hr.Avg_HR \
        FROM myrun RIGHT JOIN myrun_hr \
        ON myrun.RunDt=myrun_hr.RunDt_HR'
mycrs.execute(que)
datarequest = mycrs.fetchall()
df = pd.DataFrame(datarequest, columns=mycrs.column_names)
print('------------')
print('Length of the new table:', len(datarequest))
df

------------
Length of the new table: 387


,RunDt,RunTt,RunDs,Avg_HR
0,3/16/2018 20:48,Odesa,8.13,156
1,3/17/2018 12:23,Odesa,11.4,159
2,3/19/2018 11:37,Odesa,11.46,159
3,3/21/2018 14:03,Odesa,11.46,160
4,3/22/2018 12:02,Odesa,11.49,156
...,...,...,...,...
382,None,None,None,112
383,None,None,None,117
384,None,None,None,136
385,None,None,None,103


In [12]:
que = 'SELECT myrun.RunDt, myrun.RunTt, myrun.RunDs, myrun_hr.Avg_HR \
        FROM myrun LEFT OUTER JOIN myrun_hr \
        ON myrun.RunDt=myrun_hr.RunDt_HR \
        WHERE runDS>30'                                  # Some experements with WHERE
mycrs.execute(que)
datarequest = mycrs.fetchall()
df = pd.DataFrame(datarequest, columns=mycrs.column_names)
print('------------')
print('Length of the new table:', len(datarequest))
df

------------
Length of the new table: 11


,RunDt,RunTt,RunDs,Avg_HR
0,5/26/2019 8:40,Brasov Marathon,42.2,169
1,5/12/2019 12:01,Odesa Running,32.12,160
2,5/5/2019 14:18,Odesa Running,30.37,143
3,5/1/2019 11:19,Odesa Running,30.11,157
4,3/27/2011 0:00,Odesa,30.3,None
5,4/9/2011 0:00,Odesa Ultra Marathon,100,None
6,7/20/2014 9:12,N.Novgorod Marathon,42.2,None
7,4/10/2016 16:04,Togliatti,30.41,None
8,4/24/2016 10:00,Togliatti Marathon,42.2,None
9,5/12/2017 12:36,Palanca Running,30.84,None


## 3. GROUP BY + Aggregation functions ##

Let's go back to the table **myrun** and Group it by Location (**RunTt**)

In [13]:
que = 'SELECT RunTt, MIN(RunDs) AS RunDS_min, COUNT(RunDs) AS RunDS_COUNT, SUM(RunDs) AS RunDS_SUM \
        FROM myrun \
        GROUP BY RunTt'
mycrs.execute(que)
datarequest = mycrs.fetchall()
df = pd.DataFrame(datarequest, columns=mycrs.column_names)
print('------------')
print('Length of the new table:', len(datarequest))
df

------------
Length of the new table: 36


,RunTt,RunDS_min,RunDS_COUNT,RunDS_SUM
0,Odesa,1.99,221,2094.61
1,Odesa Ultra Marathon,100,1,100.00
2,Turkmenistan,0.03,162,1813.80
3,Desert hot trail,20.24,1,20.24
4,N.Novgorod,0.18,135,1506.32
5,N.Novgorod Marathon,42.2,1,42.20
6,Togliatti,10.04,199,2166.35
7,Togliatti Marathon,42.2,1,42.20
8,Palanca Running,1.1,117,1414.41
9,Running,10.26,12,123.29


In [14]:
que = 'SELECT RunDs, COUNT(RunTt) AS RunDS_COUNT \
        FROM myrun \
        GROUP BY RunDs \
        ORDER BY RunDs DESC'
mycrs.execute(que)
datarequest = mycrs.fetchall()
df = pd.DataFrame(datarequest, columns=mycrs.column_names)
print('------------')
print('Length of the new table:', len(datarequest))
df[:10]

------------
Length of the new table: 602


,RunDs,RunDS_COUNT
0,9.91,1
1,9.71,1
2,9.63,1
3,9.35,1
4,9.27,1
5,9.19,1
6,9.16,1
7,9.09,1
8,9.04,1
9,9.03,1


## 4. CTE (Common Table Expression) ##

In [15]:
que = 'WITH CTE (RunTt, RunDt, RunDs) AS \
        (SELECT RunTt, RunDt, RunDs \
        FROM myrun WHERE RunDs>30) \
        SELECT RunTt, RunDs, RunDt \
        FROM CTE'

mycrs.execute(que)
datarequest = mycrs.fetchall()
df = pd.DataFrame(datarequest, columns=mycrs.column_names)
print('------------')
print('Length of the new table:', len(datarequest))
df[:10]

------------
Length of the new table: 11


,RunTt,RunDs,RunDt
0,Odesa,30.3,3/27/2011 0:00
1,Odesa Ultra Marathon,100,4/9/2011 0:00
2,N.Novgorod Marathon,42.2,7/20/2014 9:12
3,Togliatti,30.41,4/10/2016 16:04
4,Togliatti Marathon,42.2,4/24/2016 10:00
5,Palanca Running,30.84,5/12/2017 12:36
6,Palanca Running,30.34,6/16/2017 18:29
7,Odesa Running,30.11,5/1/2019 11:19
8,Odesa Running,30.37,5/5/2019 14:18
9,Odesa Running,32.12,5/12/2019 12:01


In [16]:
que = 'WITH CTE (RunTt, RunDt, RunDs) AS \
        (SELECT RunTt, RunDt, Sum(RunDs) \
        FROM myrun GROUP BY RunTt) \
        SELECT RunTt, RunDs, RunDt \
        FROM CTE'

mycrs.execute(que)
datarequest = mycrs.fetchall()
df = pd.DataFrame(datarequest, columns=mycrs.column_names)
print('------------')
print('Length of the new table:', len(datarequest))
df

------------
Length of the new table: 36


,RunTt,RunDs,RunDt
0,Odesa,2094.61,1/5/2011 0:00
1,Odesa Ultra Marathon,100.00,4/9/2011 0:00
2,Turkmenistan,1813.80,6/26/2012 0:00
3,Desert hot trail,20.24,8/5/2013 19:30
4,N.Novgorod,1506.32,1/22/2014 18:34
5,N.Novgorod Marathon,42.20,7/20/2014 9:12
6,Togliatti,2166.35,4/29/2015 20:04
7,Togliatti Marathon,42.20,4/24/2016 10:00
8,Palanca Running,1414.41,1/14/2017 10:15
9,Running,123.29,3/9/2017 11:19


## 5. Bonus (RANK() vs ROW_NUMBER())

In [17]:
que = 'SELECT *, RANK() OVER(ORDER BY RunTt) Rank_NUM, \
            ROW_NUMBER() OVER(ORDER BY RunTt) Row_n \
            FROM myrun'

mycrs.execute(que)
datarequest = mycrs.fetchall()
df = pd.DataFrame(datarequest, columns=mycrs.column_names)
print('------------')
print('Length of the new table:', len(datarequest))
df

------------
Length of the new table: 1189


,id,RunDt,RunTt,RunDs,RunTm,Rank_NUM,Row_n
0,787,5/31/2017 18:30,Alexandrovka 20k,20,1:55:00,1,1
1,1134,8/26/2019 15:46,Arefu Running,1.94,0:23:21,2,2
2,1037,3/8/2019 13:34,Biliaivskyi raion - Running,5.25,0:34:32,3,3
3,1038,3/8/2019 14:10,Biliaivskyi raion - Running,2.77,0:15:42,3,4
4,1039,3/9/2019 11:48,Biliaivskyi raion - Running,7.08,0:47:13,3,5
...,...,...,...,...,...,...,...
1184,1048,3/25/2019 19:09,Yamal'skiy rayon Running,12.47,1:08:09,1116,1185
1185,1049,3/27/2019 19:14,Yamal'skiy rayon Running,12.36,1:19:33,1116,1186
1186,1050,3/29/2019 10:02,Yamal'skiy rayon Running,15.86,1:42:06,1116,1187
1187,983,10/31/2018 9:28,Yaremche Running,12.02,1:35:49,1188,1188
